In [ ]:
#https://github.com/adoku14/Eclat-Python-Implementation/blob/master/eclat.py

In [7]:
from numpy import *

In [45]:
def loadDataSet():
    return [[0, 1, 2, 3, 4, 5, 6, 7, 8], [8, 31, 32 ], [33, 32, 35 ], [3, 39, 48 ]]

It creates C1 .C1 is a candidate itemset of size one. In the Apriori algorithm, we create C1, and then we’ll scan the dataset to see if these one itemsets meet our minimum support requirements. The itemsets that do meet our minimum requirements become L1. L1 then gets combined to become C2 and C2 will get filtered to become L2.

Frozensets are sets that are frozen, which means they’re immutable; you can’t change them. You need to use the type frozenset instead of set because you’ll later use these sets as the key in a dictionary.

You can’t create a set of just one integer in Python. It needs to be a list (try it out). That’s why you create a list of single-item lists. Finally, you sort the list and then map every item in the list to frozenset() and return this list of frozensets

In [99]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

This function takes three arguments: a dataset, Ck, a list of candidate sets, and minSupport, which is the minimum support you’re interested in. This is the function you’ll use to generate L1 from C1. Additionally, this function returns a dictionary with support values.

In [47]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [48]:
dataSet = loadDataSet()
dataSet

[[0, 1, 2, 3, 4, 5, 6, 7, 8], [8, 31, 32], [33, 32, 35], [3, 39, 48]]

In [49]:
C1 = createC1(dataSet)
C1

[frozenset({0}),
 frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5}),
 frozenset({6}),
 frozenset({7}),
 frozenset({8}),
 frozenset({31}),
 frozenset({32}),
 frozenset({33}),
 frozenset({35}),
 frozenset({39}),
 frozenset({48})]

# C1 contains a list of all the items in frozenset

In [50]:
#D is a dataset in the setform.

D = list(map(set,dataSet))
D

[{0, 1, 2, 3, 4, 5, 6, 7, 8}, {8, 31, 32}, {32, 33, 35}, {3, 39, 48}]

Now that you have everything in set form, you can remove items that don’t meet our minimum support.

In [51]:
L1,suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({32}), frozenset({8}), frozenset({3})]

These four items make up our L1 list, that is, the list of one-item sets that occur in at least 50% of all transactions. Item 4 didn’t make the minimum support level, so it’s not a part of L1. That’s OK. By removing it, you’ve removed more work from when you find the list of two-item sets.

# Apriori algorithm


While the number of items in the set is greater than 0:

Create a list of candidate itemsets of length k

Scan the dataset to see if each itemset is frequent

Keep frequent itemsets to create itemsets of length k+1

The main function is apriori(); it calls aprioriGen() to create candidate itemsets: Ck.

The function aprioriGen() will take a list of frequent itemsets, Lk, and the size of the itemsets, k, to produce Ck. For example, it will take the itemsets {0}, {1}, {2} and so on and produce {0,1} {0,2}, and {1,2}.

The sets are combined using the set union, which is the | symbol in Python.

In [52]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [53]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [54]:
L,suppData = apriori(dataSet)
L

[[frozenset({32}), frozenset({8}), frozenset({3})], []]

L contains some lists of frequent itemsets that met a minimum support of 0.5. The variable suppData is a dictionary with the support values of our itemsets

In [55]:
L[0]

[frozenset({32}), frozenset({8}), frozenset({3})]

In [56]:
L[1]

[]

In [58]:
aprioriGen(L[0],2)

[frozenset({8, 32}), frozenset({3, 32}), frozenset({3, 8})]

# Mining association rules from frequent item sets

generateRules(), is the main command, which calls the other two.

The generateRules() function takes three inputs: a list of frequent itemsets, a dictionary of support data for those itemsets, and a minimum confidence threshold. It’s going to generate a list of rules with confidence values that we can sort through later.

In [59]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

calcConf() calculates the confidence of the rule and then find out the which rules meet the minimum confidence.

In [60]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

rulesFromConseq() generates more association rules from our initial dataset. This takes a frequent itemset and H, which is a list of items that could be on the right-hand side of a rule.

In [61]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [62]:
L,suppData= apriori(dataSet,minSupport=0.5)

In [64]:
rules= generateRules(L,suppData, minConf=0.7)

This gives you three rules: {1} ➞ {3},{5} ➞ {2},and {2} ➞ {5}. It’s interesting to see that the rule with 2 and 5 can be flipped around but not the rule with 1 and 3.

In [65]:
DatSet = [line.split() for line in open('retail.txt').readlines()]

In [66]:
L,suppData= apriori(DatSet, minSupport=0.3)

In [67]:
for item in L[1]:
    if item.intersection('2'):
        print (item)

In [68]:
for item in L[1]:
    if item.intersection('2'):
        print (item)

In [74]:
rules= generateRules(L,suppData, minConf=0.7)

In [75]:
rules

[]

# FP growth

In [76]:
#variables:
#name of the node, a count
#nodelink used to link similar items
#parent vaiable used to refer to the parent of the node in the tree
#node contains an empty dictionary for the children in the node
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [77]:
rootNode = treeNode('pyramid',9,None)

In [78]:
rootNode.children['eye'] = treeNode('eye',13,None)

In [79]:
rootNode.disp()

   pyramid   9
     eye   13


In [82]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table




In [83]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [84]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [85]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [86]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [87]:
simpDat = loadSimpDat()

In [88]:
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [89]:
initSet = createInitSet(simpDat)

In [90]:
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [91]:
#The FP-tree
myFPtree, myHeaderTab = createTree(initSet, 3)

In [92]:
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         t   2
           s   2
             y   2
         r   1
           t   1
             y   1
     x   1
       r   1
         s   1


In [93]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [94]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [95]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [96]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'x', 'z'}): 1}

In [9]:
import pandas as pd
import os
os.chdir(r'D:\2nd_semester\data_mining\assignment\2')

In [1]:
import numpy as np, itertools
import pandas as pd
np.random.seed(1)
kot = 0
FreqItems = dict()
support = dict()

In [24]:
minsup = 1

In [22]:
def eclat(prefix, items, dict_id):
    while items:
        i,itids = items.pop()
        isupp = len(itids)
        if isupp >= minsup:

            FreqItems[frozenset(prefix + [i])] = isupp
            suffix = []
            for j, ojtids in items:
                jtids = itids & ojtids
                if len(jtids) >= minsup:
                    suffix.append((j,jtids))

            dict_id += 1
            eclat(prefix+[i], sorted(suffix, key=lambda item: len(item[1]), reverse=True), dict_id)


In [19]:
def Read_Data(filename, delimiter=','):
    data = {}
    trans = 0
    f = open(filename, 'r', encoding="utf8")
    for row in f:
        trans += 1
        for item in row.strip().split(delimiter):
            if item not in data:
                data[item] = set()
            data[item].add(trans)
    f.close()
    return data

In [ ]:
 dict_id = 0
eclat([], sorted(data.items(), key=lambda item: len(item[1]), reverse=True), dict_id)

In [20]:
data = Read_Data('retail-small.txt', ' ')

In [21]:
data

{'0': {1},
 '1': {1},
 '2': {1},
 '3': {1, 8},
 '4': {1},
 '5': {1},
 '6': {1},
 '7': {1},
 '8': {1},
 '9': {1},
 '10': {1},
 '11': {1},
 '12': {1},
 '13': {1},
 '14': {1},
 '15': {1},
 '16': {1},
 '17': {1},
 '18': {1},
 '19': {1},
 '20': {1},
 '21': {1},
 '22': {1},
 '23': {1},
 '24': {1},
 '25': {1},
 '26': {1},
 '27': {1},
 '28': {1},
 '29': {1},
 '30': {2},
 '31': {2},
 '32': {2, 7, 10},
 '33': {3},
 '34': {3},
 '35': {3},
 '36': {4, 13, 17},
 '37': {4},
 '38': {4, 5, 6, 13, 17},
 '39': {4, 5, 6, 8, 12, 13, 16, 17},
 '40': {4},
 '41': {4, 7, 13, 15},
 '42': {4},
 '43': {4},
 '44': {4},
 '45': {4},
 '46': {4},
 '47': {5},
 '48': {5, 6, 8, 11, 13, 16, 17},
 '49': {6},
 '50': {6},
 '51': {6},
 '52': {6},
 '53': {6},
 '54': {6},
 '55': {6},
 '56': {6},
 '57': {6},
 '58': {6},
 '59': {7},
 '60': {7},
 '61': {7},
 '62': {7},
 '63': {9},
 '64': {9},
 '65': {9},
 '66': {9},
 '67': {9},
 '68': {9},
 '69': {10},
 '70': {11},
 '71': {11},
 '72': {11},
 '73': {12},
 '74': {12},
 '75': {12},
 

In [3]:
def rules(FreqItems, confidence):
    Rules = []
    cnt = 0

    for items, support in FreqItems.items():
        if (len(items) > 1):
            all_perms = list(itertools.permutations(items, len(items)))
            for lst in all_perms:
                antecedent = lst[:len(lst) - 1]
                consequent = lst[-1:]

                conf = float(FreqItems[frozenset(items)]/FreqItems[frozenset(antecedent)]*100)
                if (conf >= confidence):
                    cnt += 1
                    lift = float(conf/FreqItems[frozenset(consequent)])
                    if lift >= 1:
                        Rules.append((antecedent, consequent, support, conf, lift))


    print('Found %d Rules ' % (cnt))
    return Rules



In [4]:
def getantecendent(FreqItems, confidence):
    ant = []
    cnt = 0

    for items, support in FreqItems.items():
        if(len(items) > 1):
            all_perms = list(itertools.permutations(items, len(items)))
            for lst in all_perms:
                antecedent = lst[:len(lst) - 1]
                consequent = lst[-1:]

                conf = float(FreqItems[frozenset(items)]/FreqItems[frozenset(antecedent)]*100)
                if (conf >= confidence):
                    cnt += 1
                    lift = float(conf/FreqItems[frozenset(consequent)])
                    if lift >= 1:
                        ant.append((antecedent))

    print('Print %d attributes' % (cnt))
    return ant

def print_Frequent_Itemsets(output_FreqItems, FreqItems):
    file = open(output_FreqItems, 'w+')
    for item, support in FreqItems.items():
        file.write(" {} : {} \n".format(list(item), round(support,4)))

def print_Rules(output_Rules, Rules):
    file = open(output_Rules, 'w+')
    for a, b,supp, conf, lift in sorted(Rules):
        file.write("{} ==> {} support: {} confidence: {} \n".format((a), (b), round(supp, 4),round(conf, 4),round(lift, 4)))
    file.close()
    
def print_Antecendent(ant):
    file = open('output_antecendent.csv', 'w+')
    for a in sorted(ant):
        file.write("[] \n".format((a)))
    file.close()
    
def Read_Data(filename, delimiter=','):
    data = {}
    trans = 0
    f = open(filename, 'r', encoding="utf8")
    for row in f:
        trans += 1
        for item in row.split(delimiter):
            if item not in data:
                data[item] = set()
            data[item].add(trans)
    f.close()
    return data



In [6]:
if __name__ == "__main__":
    minsup   = 10
    confidence = 75
    output_FreqItems = 'output_freqitems.csv'
    output_Rules = 'output_rule.csv'
    dict_id = 0
    data = Read_Data('retail.txt', ',') #change the delimiter based on your input file
    data.pop("\n",None)
    data.pop("",None)
    print('finished reading data..... \n Starting mining .....')
    eclat([], sorted(data.items(), key=lambda item: len(item[1]), reverse=True), dict_id)
    print('found %d Frequent items' % len(FreqItems))
    Rules = rules(FreqItems, confidence)
    print('Writing Rules .....')



    print_Frequent_Itemsets(output_FreqItems, FreqItems)
    print_Rules(output_Rules, Rules)
    Antecendent = getantecendent(FreqItems, confidence)
    print_Antecendent(Antecendent)
    
    Ant1d = np.hstack(Antecendent)
    
    count = np.array(Ant1d)
    unique, counts = np.unique(count, return_counts=True)
    dict(zip(unique, counts))
    counted = np.stack((unique, counts), axis=1)
    appendFile = open('candidate.csv','w')
    for i in range(0,len(counted)):
        appendFile.write(str(unique[i])+";"+str(counts[i])+","+"\n")
    appendFile.close()
    
    df = pd.DataFrame(counted, columns=['word','counter'])
    df["counter"] = pd.to_numeric(df["counter"])
    sortcounted = df.sort_values(["counter"], axis=0, 
                     ascending=[False]) 
    elimcounted = sortcounted.drop(sortcounted[sortcounted['counter']<2].index)
    
    listfrequent = list(elimcounted.iloc[:, 0].values)

finished reading data..... 
 Starting mining .....
found 56 Frequent items
Found 0 Rules 
Writing Rules .....
Print 0 attributes


ValueError: need at least one array to concatenate

# Powerset

In [5]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for result in powerset([1, 2, 3]):
    print(result)

results = list(powerset([1, 2, 3]))
print(results)

()
(1,)
(2,)
(3,)
(1, 2)
(1, 3)
(2, 3)
(1, 2, 3)
[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]


In [6]:
import math
I = 'abcdef'
T = ['abc', 'acdef', 'abc', 'de']


for X in T:
    cnt = sum([1 for i in T if set(X).issubset(set(i))])
    if (cnt != 0):
        print( "(%s, %d)" % (''.join(X), cnt))

(abc, 2)
(acdef, 1)
(abc, 2)
(de, 2)


# Example: for the set S={1,2,3,4,5} how many members will the power set have?
Well, S has 5 members, so:

|P(S)| = 2^n  = 32

In [8]:
th = 2
i = 'abcdef'
D = 3

def eclat(prefix, items, D):
    if not items: return
    candidates = [(prefix | {i}, frequency(D, prefix | {i}), i)
                  for i in items if i not in prefix]
    frequent = filter(lambda x: x[1] >= th, candidates)

    for new_prefix, freq, i in frequent:
        frequent_items.append(new_prefix)
        items = items - {i}
        eclat(new_prefix, items, D)

eclat(set(), set(I), D)

NameError: name 'frequency' is not defined

In [11]:
set(['a','a','b']) | {'a'}

{'a', 'b'}

In [18]:
inp = [(set([1]),2,1)]

In [20]:
list(filter(lambda i: i[1] >= 2, inp ))

[({1}, 2, 1)]

In [ ]:
{abc,acdef,abc,de}

In [3]:
transactions = [{'a', 'd', 'e'},
{'b', 'c', 'd'},
{'a', 'c', 'e'},
{'a', 'c', 'd', 'e'},
{'a', 'e'},
{'a', 'c', 'd'},
{'b', 'c'},
{'a', 'c', 'd', 'e'},
{'b', 'c', 'e'},
{'a', 'd', 'e'}]

In [5]:
def frequency(D, inp):
    count = 0
    for i in range(len(D)):
        if inp.issubset(D[i]):
            count += 1
    return count
    

In [8]:
frequent_items = []

In [ ]:
for Ii in frequent_items:
    fi = {}
    for Ij in frequent_items:
        if Ii == frequent_items(k-1) - prefix & Ii > Ij:
            Iij = Ii U Ij
        
            

In [9]:
from orderedset import OrderedSet

th = 3
counter = 0
def eclat(prefix, items, D):
    global counter
    if not items: return
    candidates = [(prefix | OrderedSet([i]), frequency(D, prefix | {i}), i)
                  for i in items if i not in prefix]
    frequent = candidates
    frequent.sort(key = lambda x: x[2])
    #frequent.sort(key = lambda x: x[1], reverse=True)
    '''if  counter == 0:
        items = [x[2] for x in frequent]
        counter += 1
    '''
    frequent = list(filter(lambda x: x[1] >= th, candidates))
    print(frequent)
    for new_prefix, freq, i in frequent:
        frequent_items.append(new_prefix)
        try:
            items.remove(i)
        except:
            pass
        #print(items)
        #print(new_prefix)
        eclat(new_prefix, items, D)

In [10]:
eclat(OrderedSet(), ['a','b','c','d','e','f'], transactions)

[(OrderedSet(['a']), 7, 'a'), (OrderedSet(['b']), 3, 'b'), (OrderedSet(['c']), 7, 'c'), (OrderedSet(['d']), 6, 'd'), (OrderedSet(['e']), 7, 'e')]
[(OrderedSet(['a', 'c']), 4, 'c'), (OrderedSet(['a', 'd']), 5, 'd'), (OrderedSet(['a', 'e']), 6, 'e')]
[(OrderedSet(['a', 'c', 'd']), 3, 'd'), (OrderedSet(['a', 'c', 'e']), 3, 'e')]
[]
[]
[]
[]
[]
[]
[]
[]


In [183]:
{1,2,3,4,5}.intersection({1,2,3,4,5,6,7})

{1, 2, 3, 4, 5}

In [11]:
frequent_items

[OrderedSet(['a']),
 OrderedSet(['a', 'c']),
 OrderedSet(['a', 'c', 'd']),
 OrderedSet(['a', 'c', 'e']),
 OrderedSet(['a', 'd']),
 OrderedSet(['a', 'e']),
 OrderedSet(['b']),
 OrderedSet(['c']),
 OrderedSet(['d']),
 OrderedSet(['e'])]